In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import pickle



In [2]:
# %%
# Load and process the data
df = pd.read_csv('Updated_Common_Data_Tesco.csv')

# Convert date from 'YYYYMMDD' to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Extract features from date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.dayofweek

# Drop the original date column
df = df.drop(columns=['date','own_brand'])



In [3]:
df.duplicated().sum()

0

In [3]:

# %%
df.head()


,supermarket,unit_price,unit,names,category,ASDA_price,Morrisons_price,Sains_price,year,month,day,weekday
0,Tesco,11.00125,kg,Birds Eye,frozen,10.163125,9.567213,9.890,2024,1,29,0
1,Tesco,11.00125,kg,Birds Eye,frozen,10.163125,9.567213,9.890,2024,1,29,0
2,Tesco,4.20000,kg,Heinz Oxtail Soup,food_cupboard,4.200000,4.200000,4.300,2024,1,29,0
3,Tesco,10.50000,l,Lea & Perrins Worcestershire Sauce,food_cupboard,11.300000,11.300000,10.150,2024,1,29,0
4,Tesco,3.90000,kg,Heinz Tomato Ketchup,food_cupboard,4.800000,5.666667,5.425,2024,1,29,0


In [4]:

# Encode categorical features
label_encoders = {}
for column in ['supermarket', 'names','unit','category']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Function to create sequences for LSTM
def create_sequences(group, sequence_length):
    X, y = [], []
    for i in range(len(group) - sequence_length):
        X.append(group.iloc[i:i + sequence_length].drop(columns='unit_price').values)
        y.append(group.iloc[i + sequence_length]['unit_price'])
    return np.array(X), np.array(y)

sequence_length = 10
X_list, y_list = [], []

# Group by the product or other unique identifier column
for _, group in df.groupby('names'):
    X_group, y_group = create_sequences(group, sequence_length)
    X_list.append(X_group)
    y_list.append(y_group)

X = np.concatenate(X_list)
y = np.concatenate(y_list)


X = X.astype(np.float32)
y = y.astype(np.float32)

In [5]:


# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure correct data types for train and test sets
train_X = train_X.astype(np.float32)
train_y = train_y.astype(np.float32)
test_X = test_X.astype(np.float32)
test_y = test_y.astype(np.float32)

# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Build the enhanced LSTM model with additional layers
model = Sequential()

# First LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))

# Second LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Fourth LSTM layer with Dropout
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))

# First Dense layer
model.add(Dense(25, activation='relu'))

# Second Dense layer
model.add(Dense(10, activation='relu'))

# Output layer
model.add(Dense(1))

# Compile the model with the custom RMSE loss function
model.compile(optimizer='adam', loss=rmse)

# Train the model
model.fit(train_X, train_y, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model
loss = model.evaluate(test_X, test_y)
print(f"Test RMSE: {loss}")



Epoch 1/100
163/163 [==============================] - 18s 51ms/step - loss: 298.1140 - val_loss: 302.5584
Epoch 2/100
163/163 [==============================] - 5s 34ms/step - loss: 288.2452 - val_loss: 303.3115
Epoch 3/100
163/163 [==============================] - 8s 47ms/step - loss: 297.5386 - val_loss: 302.7641
Epoch 4/100
163/163 [==============================] - 6s 35ms/step - loss: 294.5920 - val_loss: 302.5957
Epoch 5/100
163/163 [==============================] - 5s 33ms/step - loss: 292.6955 - val_loss: 302.4913
Epoch 6/100
163/163 [==============================] - 5s 32ms/step - loss: 292.1523 - val_loss: 302.5486
Epoch 7/100
163/163 [==============================] - 6s 34ms/step - loss: 284.4915 - val_loss: 303.1962
Epoch 8/100
163/163 [==============================] - 5s 33ms/step - loss: 294.8849 - val_loss: 302.5580
Epoch 9/100
163/163 [==============================] - 5s 32ms/step - loss: 293.7898 - val_loss: 302.6652
Epoch 10/100
163/163 [=======================

In [6]:

# Save the model
model.save('Tesco_lstm_model1.h5')


# Load the model
model = load_model('Tesco_lstm_model1.h5',compile=False)


import pickle

# Save label encoders
with open('Tesco_label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


# Load label encoders
with open('Tesco_label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)



model.predict(test_X)


array([[10.621125],
       [20.739857],
       [20.996904],
       ...,
       [20.990881],
       [20.830196],
       [20.775818]], dtype=float32)